In [107]:
#Kiwi Randolph
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from scipy.spatial.distance import cdist
import numpy as np 

In [61]:
trainSIFT = np.load('train_D-1.npy', allow_pickle=True)
testSIFT = np.load('test_D-1.npy', allow_pickle=True)
print(trainSIFT.shape)

(1888,)


In [ ]:
sift_train_concatenated = np.concatenate(trainSIFT, axis=0)
sift_test_concatenated = np.concatenate(testSIFT, axis=0)
clusterTrain, labelTrain = kmeans_clustering(150, sift_train_concatenated)
clusterTest, labelTest = kmeans_clustering(150, sift_test_concatenated)

In [108]:
def kmeans_clustering(cluster, trainf):
    kmeans = KMeans(n_clusters = cluster, random_state = 0, init = 'k-means++', n_init = 15, max_iter=100)

    kvalue = kmeans.fit(trainf)
    labels = kmeans.predict(trainf)
    
    cluster = kmeans.cluster_centers_
    
    return cluster, labels

# def build_bow_representation(sift_features, labels, cluster_centers):
#     bow_representation = []
#     for i, features in enumerate(sift_features):
#         bow_vector = np.bincount(labels[i], minlength=len(cluster_centers))
#         bow_representation.append(bow_vector)
#     return np.array(bow_representation)

def build_bow_features(data, visual_words):
    bow_features = []
    for descriptors in data:
        distances = cdist(descriptors, visual_words)
        nearest_visual_words = np.argmin(distances, axis=1)
        histogram, _ = np.histogram(nearest_visual_words, bins=range(len(visual_words) + 1))
        bow_features.append(histogram)
    return np.array(bow_features)


def kNN(train_bow_list, trainingLabels, test_bow_list, n_neighbors):
    knn_model = KNeighborsClassifier(n_neighbors=n_neighbors)
    
    knn_model.fit(train_bow_list, trainingLabels)
    
    predictions = knn_model.predict(test_bow_list)
    
    return predictions

In [109]:
trainLabel = np.load('train_gs-1.npy')
testLabel = np.load('test_gs.npy')
trainLabel = np.load('train_gs-1.npy').reshape(-1)
testLabel = np.load('test_gs.npy').reshape(-1)
print(trainLabel.shape)
print(testLabel.shape)
labelTrain_reshaped = np.array_split(labelTrain, len(trainSIFT))
labelTest_reshaped = np.array_split(labelTest, len(testSIFT))

# BOWtrain = build_bow_representation(trainSIFT, labelTrain_reshaped, clusterTrain)
# BOWtest = build_bow_representation(testSIFT, labelTest_reshaped, clusterTest)
BOWtrain = build_bow_features(trainSIFT, clusterTrain)
BOWtest = build_bow_features(testSIFT, clusterTest)

# BOWtrain = assign_bow(trainSIFT, clusterTrain, trainLabel)
# BOWtest = assign_bow(testSIFT, clusterTest, testLabel)
# print(BOWtrain)
# print(BOWtest)


(1888,)
(800,)


In [110]:
predictions = kNN(BOWtrain, trainLabel, BOWtest, n_neighbors=20)


# # Flatten the arrays for compatibility
# testLabel_flat = testLabel.flatten()
# predicted_labels_flat = predicted_labels.flatten()



accuracy = accuracy_score(testLabel, predictions)
print("Accuracy:", (accuracy * 100), "%")
conf_matrix = confusion_matrix(testLabel, predictions)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 15.875 %
Confusion Matrix:
[[ 1 31  0 21  2  4 12 29]
 [ 6  0 13  2 14 32 24  9]
 [ 0 21  0 20  3  6 21 29]
 [ 1  1  2 11 12 37 25 11]
 [ 0  3  0 14 10  9 49 15]
 [ 1 15  0 14  3 10 34 23]
 [ 0  2  1  7 10 11 65  4]
 [ 0  6  1 11 10 16 26 30]]
